In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import openpyxl as op

In [2]:
# Settings :

# from light_sim.ipynb
# Settings angles
#####
start_angle = -60
end_angle = 60
step_angle = 5
angles = np.arange(start_angle, end_angle + 1, step_angle)
angles = [-60]
print(list(angles))
titres_colonnes = np.append(angles, np.array("temoin", dtype=object), axis=None)

# Annual parameters
###### # from light_sim.ipynb

######
# Settings, year and hours
HEURE_DEB = 10
HEURE_FIN = 14
start_date = datetime(year=2023, month=1, day=1, hour=HEURE_DEB)  # Début de l'année (éviter les années bissextiles)
end_date = datetime(year=2024, month=1, day=1, hour=HEURE_DEB)  # Fin de l'année (début de l'année suivante)
NB_HEURES_ANNEES = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days
HEURE_BLANCHES = 24 - (HEURE_FIN-HEURE_DEB)
NB_HEURES_ANNEES_ETUD = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days
TOT_HOYS = (end_date-start_date).days*24

[-60]


In [3]:
# Read the generated table in previous steps
df = pd.read_excel("results/10-14H_-60_classic_fin1.xlsx",sheet_name="Sheet1",header=None)
df.columns = titres_colonnes
print(df)

           -60    temoin
0     0.024941  0.043242
1     0.058469  0.085575
2     0.139705  0.167588
3     0.149662  0.229777
4     0.029182  0.051969
...        ...       ...
1455  0.374994  0.543843
1456  0.275236  0.540999
1457  0.436942  0.637138
1458  0.511633  0.638061
1459  0.365061  0.518911

[1460 rows x 2 columns]


In [4]:
# Step 1: Add as many null lines as HEURE_DEB if positive
if HEURE_DEB > 0:
    null_rows_start = pd.DataFrame(0, index=range(HEURE_DEB), columns=titres_colonnes)
    df2 = pd.concat([null_rows_start, df], ignore_index=True)

# Step 2: Add 8 empty lines every 16 lines until 8760 lines
if HEURE_BLANCHES > 0:
    empty_rows = pd.DataFrame(0, index=range(HEURE_BLANCHES), columns=titres_colonnes)

    for idx in range(HEURE_FIN, TOT_HOYS, 24):
        df2 = pd.concat([df2.iloc[:idx], empty_rows, df2.iloc[idx:]], ignore_index=True)

# Step 3: Remove excess lines if any
if df2.shape[0] > TOT_HOYS :
    surplus = df2.shape[0] - TOT_HOYS
    df3 = df2.iloc[:-surplus]

In [5]:
print(df3)

      -60  temoin
0     0.0     0.0
1     0.0     0.0
2     0.0     0.0
3     0.0     0.0
4     0.0     0.0
...   ...     ...
8755  0.0     0.0
8756  0.0     0.0
8757  0.0     0.0
8758  0.0     0.0
8759  0.0     0.0

[8760 rows x 2 columns]


In [6]:
# Create a DataFrame with the hours of the year (from 0 to 8759) as an index
hours_in_year = range(8760)
dfX = pd.DataFrame(index=hours_in_year)

# Add the 'month_day_hour' column
df3['month_day_hour'] = pd.to_datetime(dfX.index.map(lambda x: start_date + timedelta(hours=x - HEURE_DEB)))

# Rearrange columns in df3
cols = ['month_day_hour'] + [col for col in df3 if col != 'month_day_hour']
df3 = df3.loc[:, cols]
print(df3.head())

       month_day_hour  -60  temoin
0 2023-01-01 00:00:00  0.0     0.0
1 2023-01-01 01:00:00  0.0     0.0
2 2023-01-01 02:00:00  0.0     0.0
3 2023-01-01 03:00:00  0.0     0.0
4 2023-01-01 04:00:00  0.0     0.0


c:\Users\maceo.valente\.virtualenvs\TSAgriPV-XqNYGekU\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df3.to_excel("bdd_irr/10-14H_-60_classic_fin1.xlsx", index=False, header=True)

Import scenario :
1 = True = Backtracking (productible max)
0 = False = Effacement (irradiance au sol maximum)

In [43]:
from scenario import Scenario

## TESTS ##
# Instancier des objets scénarios
s_10pc = Scenario(file_path="scenarios/sc_10%.xlsx")
s_full_BT = Scenario(file_path="scenarios/sc_full_BT.xlsx")
# print(s1.scenario[:,:])

print(s_10pc.scenario[:,:])
print(s_10pc.count_erased_hours())

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
22


In [44]:
print(s_10pc)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


Création tab_1

In [45]:
# Nombre de lignes souhaité
n_rows = 8760

# Création d'un DataFrame avec des valeurs par défaut (par exemple, 0)
tab_1 = pd.DataFrame({
    'hoys': df3['month_day_hour'],   #Hours of the Year
    'effacement': np.ones(n_rows),   #Panel priority mode, 1 = priority to
    'angle': np.zeros(n_rows),
    'irradiance_sol': np.zeros(n_rows)
})


Import PVSyst

In [46]:
# Chemin vers le fichier Excel
file_path = 'PIOLENC_PVSyst_output.xlsm'

# Nom de l'onglet à lire
sheet_names = ['Tests','Illim']

# Lire l'onglet spécifique
pvsyst = pd.read_excel(file_path, sheet_name=sheet_names)

In [47]:
pvsyst_illim = pvsyst['Illim'].drop(index=list(range(12))).drop(columns=['Unnamed: 4', 'Unnamed: 5']).reset_index(drop=True)
pvsyst_illim.columns = ['Date','E_Grid','PR','PhiAng']
print(pvsyst_illim)

                     Date  E_Grid PR PhiAng
0     1990-01-01 00:00:00 -0.0132  0      0
1     1990-01-01 01:00:00 -0.0132  0      0
2     1990-01-01 02:00:00 -0.0132  0      0
3     1990-01-01 03:00:00 -0.0132  0      0
4     1990-01-01 04:00:00 -0.0132  0      0
...                   ...     ... ..    ...
8755  1990-12-31 19:00:00 -0.0132  0      0
8756  1990-12-31 20:00:00 -0.0132  0      0
8757  1990-12-31 21:00:00 -0.0132  0      0
8758  1990-12-31 22:00:00 -0.0132  0      0
8759  1990-12-31 23:00:00 -0.0132  0      0

[8760 rows x 4 columns]


In [48]:
pvsyst_tests = pvsyst['Tests'].drop(index=list(range(12))).drop(columns=pvsyst['Tests'].columns[4:]).reset_index(drop=True)
pvsyst_tests.columns = ['Date', 'PhiAng', 'E_Grid_SunTrack', 'Egrid Effectif']
print(pvsyst_tests)

                     Date PhiAng E_Grid_SunTrack Egrid Effectif
0     1990-01-01 00:00:00      0         -0.0132              1
1     1990-01-01 01:00:00      0         -0.0132              1
2     1990-01-01 02:00:00      0         -0.0132              1
3     1990-01-01 03:00:00      0         -0.0132              1
4     1990-01-01 04:00:00      0         -0.0132              1
...                   ...    ...             ...            ...
8755  1990-12-31 19:00:00      0         -0.0132             12
8756  1990-12-31 20:00:00      0         -0.0132             12
8757  1990-12-31 21:00:00      0         -0.0132             12
8758  1990-12-31 22:00:00      0         -0.0132             12
8759  1990-12-31 23:00:00      0         -0.0132             12

[8760 rows x 4 columns]


Remplir le tab_1 :

In [49]:
# Remplissage de la colonne 'effacement'
# Parcourir scénario, pour chaque case 0 (False), changer les 1 (panneaux en suntracking) de la colonne 'effacement' en 0 (panneaux effacés)

for i in range(s_10pc.scenario.shape[0]) :  # Pour chaque ligne
    for j in range(s_10pc.scenario.shape[1]) :  # Pour chaque colonne
        if s_10pc.scenario[i,j] == False :
            tab_1.loc[(tab_1['hoys'].dt.hour == j) & (tab_1['hoys'].dt.month == i+1), 'effacement'] = 0

In [52]:
# Remplissage de la colonne 'angle'
# Parcourir tableau_1, en fonction de 'effacement', remplir 'angle' et 'irradiance'
for i in range(tab_1.shape[0]) :
    if tab_1.at[i,'effacement'] == 0:
        #aller chercher irradiance au sol maximum dans tab_final (df3 dans ce .ipynb), et retrouver l'angle correspondant (nom de colonne)
        tab_1.loc[i,'irradiance_sol'] = df3.drop(columns=['month_day_hour','temoin']).iloc[i].max() # Irradiance max
        tab_1.loc[i,'angle'] = df3.drop(columns=['month_day_hour','temoin']).iloc[i].idxmax() # Colonne dont l'irradiance est max
        # /!\ Si égalité entre plusieurs angles, va choisir l'angle le moins élevé (-60 à priori)
    else: #'effacement' == 1
        # pvsyst_illim.loc[...] = angle PVSYST optimal, arrondir, puis trouver irradiance au sol correspondante
        tab_1.loc[i,'irradiance_sol'] = df3.at[i,round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle]
        tab_1.loc[i,'angle'] = round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle

# Add control values to tab_1 :

tab_1['temoin'] = df3.loc[:,'temoin']
# Tableau_1 rempli, il ne reste qu'à calculer les rapports entre zone témoin /!\(à intégrer)/!\ et zone d'étude
  

In [66]:
tab_1[0:24]

,hoys,effacement,angle,irradiance_sol,temoin
0,2023-01-01 00:00:00,1.0,0.0,0.000000,0.000000
1,2023-01-01 01:00:00,1.0,0.0,0.000000,0.000000
2,2023-01-01 02:00:00,1.0,0.0,0.000000,0.000000
3,2023-01-01 03:00:00,1.0,0.0,0.000000,0.000000
4,2023-01-01 04:00:00,1.0,0.0,0.000000,0.000000
5,2023-01-01 05:00:00,1.0,0.0,0.000000,0.000000
6,2023-01-01 06:00:00,1.0,0.0,0.000000,0.000000
7,2023-01-01 07:00:00,1.0,0.0,0.001722,0.002280
8,2023-01-01 08:00:00,1.0,-5.0,0.009449,0.012313
9,2023-01-01 09:00:00,1.0,-15.0,0.021566,0.026178


In [35]:
# tab_1.to_excel("step5_semitransp_10%.xlsx", index=False, header=True)

Calcul liste de taille 365 concernant rapport de irradiance_zone_témoin sur irradiance_zone_étude

In [67]:
df_daily = tab_1.groupby(tab_1['hoys'].dt.date)['irradiance_sol','temoin'].sum().reset_index()

df_daily.columns = ['date', 'irradiance_sol_day','temoin_day']
print(df_daily)

           date  irradiance_sol_day  temoin_day
0    2023-01-01            0.645644    0.871252
1    2023-01-02            0.324559    0.415251
2    2023-01-03            1.050197    1.295170
3    2023-01-04            2.517704    3.137864
4    2023-01-05            2.811059    3.467914
..          ...                 ...         ...
360  2023-12-27            2.504803    3.308104
361  2023-12-28            0.593889    0.765175
362  2023-12-29            2.021913    2.567961
363  2023-12-30            2.692507    3.501543
364  2023-12-31            2.705956    3.343441

[365 rows x 3 columns]


c:\Users\maceo.valente\.virtualenvs\TSAgriPV-XqNYGekU\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [75]:
#Export to an excel the daily ratio between control radiation and study-zone radiation

daily_ratio = df_daily['irradiance_sol_day']/df_daily['temoin_day']
# daily_ratio.to_excel('daily_ratio_rad.xlsx',header=False,index=False)
print(daily_ratio)

#Daily_ratio are to be multiplied with the Rad column of the climate STICS files

0      0.741054
1      0.781598
2      0.810856
3      0.802363
4      0.810591
         ...   
360    0.757172
361    0.776148
362    0.787361
363    0.768949
364    0.809333
Length: 365, dtype: float64
